<a href="https://colab.research.google.com/github/KinzaaSheikh/lm_research_notes/blob/main/Hands_On_LLM_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## My Notebook of Hands on LLM book

# Chapter 1

Introduced the recent history of Large Language Models and ended with a small coding example. Its interesting for me to note that a simple query-answer workflow with an LLM is intuitive enough even without using any of the recent frameworks.

In [1]:
!pip install -Uq transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 116.2 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=False,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [3]:
from transformers import pipeline

# Create a pipeline
generation = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=500,
    do_sample=False
    )

Device set to use cuda
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [4]:
# The prompt (user input / query)
messages = [
    {
        "role": "user",
        "content": "Tell me a funny joke about chickens"
    }
]

# Generate output
output = generation(messages)
print(output[0]["generated_text"])

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


 Why did the chicken join the band? Because it had the drumsticks!


# Chapter 2



### Tokenization & Embeddings

The two main pillars of LLM

Tokenization: The smallest chunk a text can be broken down to.


Embeddings: The act of converting those tokens into computable language

In [5]:
!pip install -Uq transformers

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# load model and tokenizer just like before
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=False,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

In [6]:
prompt = "Write an email to my advisor explaining why I couldn't finish proposal on time. Explain how it happend. <|assistant|>"

# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

# Generate the text
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=20
)

# Print the output
print(tokenizer.decode(generation_output[0]))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Write an email to my advisor explaining why I couldn't finish proposal on time. Explain how it happend. <|assistant|> Subject: Request for Extension on Proposal Submission

Dear Professor [Ad


In [7]:
# Print to see what's inside input_ids

print(input_ids)

tensor([[14350,   385,  4876,   304,   590,   594, 19188, 24232,  2020,   306,
          8496, 29915, 29873,  8341, 24963,   373,   931, 29889, 12027,  7420,
           920,   372,  2250,   355, 29889, 29871, 32001]], device='cuda:0')


In [8]:
# Inspect input ids using tokenizer's decode method
# translate the id's back into human readable text

for id in input_ids[0]:
  print(tokenizer.decode(id))

Write
an
email
to
my
ad
visor
explaining
why
I
couldn
'
t
finish
proposal
on
time
.
Exp
lain
how
it
happ
end
.

<|assistant|>


In [9]:
print(generation_output)

tensor([[14350,   385,  4876,   304,   590,   594, 19188, 24232,  2020,   306,
          8496, 29915, 29873,  8341, 24963,   373,   931, 29889, 12027,  7420,
           920,   372,  2250,   355, 29889, 29871, 32001,  3323,   622, 29901,
         10729,   363,  7338,  2673,   373,  1019,  1066,   284,  3323,  6737,
            13,    13, 29928,   799, 11386,   518,  3253]], device='cuda:0')


In [10]:
# we can decode the tokenizer on the output side to translate the token id in actual text
print(tokenizer.decode(8496))
print(tokenizer.decode(29915))
print(tokenizer.decode(29873))

couldn
'
t


In [11]:
print(tokenizer.decode([8496, 29915, 29873]))

couldn't


In [13]:
# Generate contextualized word embeddings

from transformers import AutoModel, AutoTokenizer

# Load a tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")

# Load a language model
model = AutoModel.from_pretrained("microsoft/deberta-v3-xsmall")

# Tokenize the sentence
tokens = tokenizer('Hello World', return_tensors='pt')

# Process the toekns
output = model(**tokens)[0]

In [14]:
output.shape

torch.Size([1, 4, 384])

In [15]:
# Inspect why there are four tokens in two words

for token in tokens['input_ids'][0]:
  print(tokenizer.decode(token))

[CLS]
Hello
 World
[SEP]


In [16]:
print(output)

tensor([[[-3.2520,  0.1818, -0.1254,  ..., -0.0502, -0.2334,  0.8897],
         [-0.4673,  0.1730, -0.0206,  ..., -0.5289,  0.7303,  2.1177],
         [-0.4950,  0.0564,  0.2842,  ...,  1.0543, -0.1747,  1.3793],
         [-2.9601,  0.2129, -0.1138,  ...,  0.1518, -0.2094,  1.0494]]],
       grad_fn=<NativeLayerNormBackward0>)


In [17]:
# Text embeddings
from sentence_transformers import SentenceTransformer

# Load model
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

# Convert text to text embeddings
vector = model.encode("Best movie ever!")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [18]:
vector.shape

(768,)

### Word embeddings beyond LLMs


NOTE: Gensim causes a lot of dependency errors so its best to kill the runtime and start over from here

In [4]:
!pip -q install gensim

In [5]:
# Download embeddings (66MB, glove, trained on wikipedia, vector size: 50)
# Other options include "word2vec-google-news-300"
# More options at https://github.com/RaRe-Technologies/gensim-data
# Installing specific versions to avoid conflicts
import gensim.downloader as api
model = api.load("glove-wiki-gigaword-50")

In [6]:
model.most_similar([model['king']], topn=11)

[('king', 1.0000001192092896),
 ('prince', 0.8236179351806641),
 ('queen', 0.7839043140411377),
 ('ii', 0.7746230363845825),
 ('emperor', 0.7736247777938843),
 ('son', 0.766719400882721),
 ('uncle', 0.7627150416374207),
 ('kingdom', 0.7542161345481873),
 ('throne', 0.7539914846420288),
 ('brother', 0.7492411136627197),
 ('ruler', 0.7434253692626953)]

### Training a song embedding model

In [7]:
import pandas as pd
from urllib import request

# Get the playlist dataset file
data = request.urlopen("https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt")

# Parse the playlist dataset file. Skip the first to lines as
# they only contian metadata
lines = data.read().decode("utf-8").split('\n')[2:]

# Remove playlists with only song
playlists = [s.rstrip().split() for s in lines if len(s.split()) > 1]

# Load song metadata
songs_file = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt')
songs_file = songs_file.read().decode('utf-8').split('\n')
songs = [s.rstrip().split('\t') for s in songs_file]
songs_df = pd.DataFrame(data=songs, columns = ['id', 'title', 'artist'])
songs_df = songs_df.set_index('id')

In [8]:
print('Playlist #1:/n', playlists[0]), '\n'

Playlist #1:/n ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '2', '42', '43', '44', '45', '46', '47', '48', '20', '49', '8', '50', '51', '52', '53', '54', '55', '56', '57', '25', '58', '59', '60', '61', '62', '3', '63', '64', '65', '66', '46', '47', '67', '2', '48', '68', '69', '70', '57', '50', '71', '72', '53', '73', '25', '74', '59', '20', '46', '75', '76', '77', '59', '20', '43']


(None, '\n')

In [9]:
# Train the model
from gensim.models import Word2Vec

# Train our Word2Vec model
model = Word2Vec(
    playlists,
    vector_size=32,
    window=20,
    negative=50,
    min_count=1,
    workers=4
)

In [10]:
song_id = 2172

# Ask the model for songs similar to song number 2172
model.wv.most_similar(positive=str(song_id))

[('2849', 0.9970338940620422),
 ('3116', 0.9966519474983215),
 ('2976', 0.9958617091178894),
 ('6626', 0.9958308339118958),
 ('3126', 0.995164155960083),
 ('11517', 0.9950485229492188),
 ('6658', 0.9949426054954529),
 ('11502', 0.994185209274292),
 ('3136', 0.9938774704933167),
 ('2640', 0.9937332272529602)]

In [11]:
print(songs_df.iloc[2173])

title                      Freedom
artist    Rage Against The Machine
Name: 2173 , dtype: object


In [12]:
# Results are all heavy metal and hard rock, within the same genre
import numpy as np

def print_recommendations(song_id):
  similar_songs = np.array(
      model.wv.most_similar(positive=str(song_id), topn=5)
  )[:,0]

  return songs_df.iloc[similar_songs]

# Extract recommendations
print_recommendations(2172)

,title,artist
id,,
2849,Run To The Hills,Iron Maiden
3116,Communication Breakdown,Led Zeppelin
2976,I Don't Know,Ozzy Osbourne
6626,Blackout,Scorpions
3126,Heavy Metal,Sammy Hagar


# Chapter 3

NOTE: Rerun the first 3 cells of the notebook
